In [1]:
import ftplib
import time as tm
import os

In [2]:
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import to_categorical
import librosa
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import IPython.display as ipd
from twilio.rest import Client

In [3]:
SERVER = 'ftp-serveralways.alwaysdata.net'
USER = 'serveralways'
USERPASS = '#####'

SID = 'ACdd02d37c618e196bf7d73790984b4797'
TOKEN = '#####'

WSP_TO = '###'
WSP_FROM = '#####'

In [5]:
# funcion de prediccion
def model_Prediction(file_name):
    # carga el erchivo
    audio_data, sample_rate = librosa.load(file_name, res_type="kaiser_fast")
    # obtiene las caracteristicas MFCC
    feature = librosa.feature.mfcc(y=audio_data, sr=sample_rate, n_mfcc=128)
    # escalado
    feature_scaled = np.mean(feature.T, axis=0)
    # lista las caracteristicas
    prediction_feature = np.array([feature_scaled])
    # redimenciona
    final_prediction_feature = prediction_feature.reshape(
        prediction_feature.shape[0], 16, 8, 1
    )
    # obtiene la id de la clase usando argmax
    predicted_vector = np.argmax(model.predict(final_prediction_feature), axis=-1)
    #obtiene la categoria de la clase desde la id
    predicted_class = le.inverse_transform(predicted_vector)
    # resultado
    print("El modelo a predicho la clase como  --> ", predicted_class[0])

    return str(predicted_class[0])

In [9]:
sid = SID
authToken = TOKEN
client = Client(sid, authToken)

model = load_model('Model.h5')

final = pd.read_pickle("extracted_df.pkl")
y = np.array(final["class"].tolist())
le = LabelEncoder()
to_categorical(le.fit_transform(y))

ftp = ftplib.FTP(SERVER, USER, USERPASS)
ftp.pwd()
ftp.cwd("audios")
ftp.retrlines('LIST')

while True:
   try:
      archivos = ftp.nlst()
      num = len(archivos)

      if num == 0:
         print("Directorio del servidor vacio, esperando archivos wav")
         tm.sleep(5)
         continue
      else:
         file = archivos[0]

         ftp.sendcmd("TYPE I")
         size = ftp.size(archivos[0])

         while size != 160044:
            tm.sleep(1)

         tiempo = tm.localtime()
         audio_name = 'mic_'+str(tiempo[0])+str("%02d" % tiempo[1])+str("%02d" % tiempo[2])+'_'+str("%02d" % tiempo[3])+'_'+str("%02d" % tiempo[4])+'_'+str("%02d" % tiempo[5])+'.wav'
         
         destino = 'K:/Proyecto T/Audios_server/'+ audio_name

         with open(destino, 'wb') as f:
            ftp.retrbinary('RETR ' + file, f.write)

         ftp.delete(file)

         prediction = model_Prediction(destino)
         # play the file
         # ipd.Audio(destino)

         if prediction == "Grito" or prediction == "Llorar" :
            mensaje = client.messages.create(to='whatsapp:'+WSP_TO, from_='whatsapp:'+WSP_FROM, body='Alerta desde dispositivo Esp32')
            print(prediction+" identificado, mensaje enviado")
            tm.sleep(2)
         else:
            print(prediction+" identificado, sin acción nesesaria")
            tm.sleep(2)
   except:
      print("Error encontrado en ciclo while")
      break

-rw-rw-r--   1 serveralways 337638     160044 Jan 16 01:22 mic_20240114_15_13_55.wav
-rw-rw-r--   1 serveralways 337638     160044 Jan 16 01:22 mic_20240116_00_31_58.wav
-rw-rw-r--   1 serveralways 337638     160044 Jan 16 01:22 mic_20240116_00_32_17.wav
-rw-rw-r--   1 serveralways 337638     160044 Jan 16 01:22 mic_20240116_00_41_25.wav
-rw-rw-r--   1 serveralways 337638     160044 Jan 16 01:18 mic_20240116_00_41_44.wav
-rw-rw-r--   1 serveralways 337638     160044 Jan 16 01:18 mic_20240116_00_42_02.wav
-rw-rw-r--   1 serveralways 337638     160044 Jan 16 01:18 mic_20240116_00_42_20.wav
-rw-rw-r--   1 serveralways 337638     160044 Jan 16 01:18 mic_20240116_00_42_39.wav
1/1 [==============================] - 0s 58ms/step
El modelo a predicho la clase como  -->  Grito
Grito identificado, mensaje enviado
1/1 [==============================] - 0s 17ms/step
El modelo a predicho la clase como  -->  Ladrido
Ladrido identificado, sin acción nesesaria
1/1 [==============================] - 0s